# Chapter12. 내장 자료형 상속과 다중 상속
- 내장 자료형 상속의 위험성
- 다중 상속과 메서드 결정 순서  

Tkinter GUI toolkit과 Django를 통해 다중 상속을 설명한다.

## 12.1 내장 자료형의 상속은 까다롭다
python 2.2 이전까지는 list나 dict 등 내장 자료형을 상속할 수 없었다.

In [2]:
#[12-1] __setattr__()을 오버라이드한 메서드를 무시하는 내장된 __init__()과, __update__() 메서드\

class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [3]:
dd = DoppelDict(one=1)  # init은 setitem이 오버라이딩 된 것을 무시함
dd

{'one': 1}

In [13]:
dd['two']=2     # []연산자는 오버라이드한 __setitem__()을 호출하므로 two가 복제된 [2,2]에 매핑됨
dd

{'one': 1, 'two': [2, 2], 'three': 2}

In [15]:
dd.update(three= 3)     # dict클래스의 update 메서드도 __setitem__()을 호출하지 않으므로 복제되지 않는다
dd

{'one': 1, 'two': [2, 2], 'three': 3}

In [16]:
#[12-2] AnswerDict의 __getitem__()을 지나지는 dict.update()
class AnswerDict(dict):
    def __getitem__(self, key):     # key와 무관하게 42 리턴
        return 42

In [17]:
ad = AnswerDict(a='foo')
ad['a']     # getitem() returns

42

In [19]:
d = {}
d.update(ad)    # ad객체를 이용해 갱신
d['a']          # dict.update()는 AnswerDict.__getitem__()을 무시한다.

'foo'

In [20]:
d

{'a': 'foo'}

In [21]:
#[12-3] dict이 아니라 UserDict을 상속하면 위와 같은 문제가 해결된다.
import collections

class DoppelDict2(collections.UserDict):
    def __setitem__(self, key, value):
        return super().__setitem__(key, [value] * 2)

class AnswerDict2(collections.UserDict):
    def __getitem__(self, key):
        return 42

In [23]:
dd = DoppelDict2(one=1)
dd

{'one': [1, 1]}

In [24]:
dd['two']=2
dd

{'one': [1, 1], 'two': [2, 2]}

In [26]:
dd.update(three=3)
dd

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}

In [29]:
ad = AnswerDict2(a='foo')
ad['a']

42

In [30]:
d = {}
d.update(ad)
d['a']

42

In [31]:
d

{'a': 42}

위 문제는 C언어로 구현된 내장 자료형의 메서드에 위임할 때만 발생하므로, 이러한 내장 자료형을 상곡한 사용자 정의 클래스에만 영향을 미친다. 따라서 UserDict이나, MutableMapping 등 파이썬으로 구현된 클래스를 상속할 때는 이런 문제가 발생하지 않는다.

## 12.2 다중 상속과 메서드 결정 순서
다중 상속을 지원하는 언어에서는 별개의 상위클래스가 동일한 이름으로 메서드를 구현할 때 발생하는 이름 충돌 문제를 해결해야 한다. 예제[12-4]에서 보여주는 이름 충돌 문제를 '다이아몬드 문제'라고 한다.

In [60]:
#[12-4] 그림 [12-1]을 구현한 ABCD클래스
class A:
    def ping(self):
        print('ping:', self)

class B(A):
    def pong(self):
        print('pong:', self)

class C(A):
    def pong(self):
        print('PONG:', self)

class D(B, C):
    def ping(self):
        super().ping()
        print('post-ping:', self)
    
    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()

In [49]:
from diamond import *

d = D()
d.pong()        # d.pong() -> Bz크래스 메서드 호출
C.pong(d)       # 객체를 인수로 전달하면 슈퍼클래스의 메서드를 직접 호출할 수 있다.

pong: <diamond.D object at 0x1065f9400>
PONG: <diamond.D object at 0x1065f9400>


파이썬이 상속 그래프를 조회할 떄는 특정한 순서를 따르므로, 위와 같은 호출의 모호함이 해결된다. 이 순서를 메서드 결정 순서(MRO; Method Resolution Order)라고 한다. 클래스에 있는 `__mro__` 속성은 현재 클래스부터 object 클래스까지 슈퍼클래스들의 MRO를 튜플 형태로 저장한다.

In [50]:
D.__mro__

(diamond.D, diamond.B, diamond.C, diamond.A, object)

슈퍼클래스 메서드에 위임할 떄는 내장 함수인 super()를 사용하는 것이 좋다. D 클래스 pingpong()메서드 처럼 파이썬 3에서는 사용하기가 더 쉬워졌다. 그러나 MRO를 위회해서 슈퍼클래스 메서드를 직접 호출할 수 있으며, 때로는 이 방법이 더 편리하다. 
```python
class D(B, C):
    def ping(self):     # super().ping() 대신 호출
        print('post-ping:', self)
    
    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
```

In [63]:
# 객체 메서드를 클래스에 직접 호출할 때는 self를 반드시 명시해야 한다. 바인딩되지 않는 메서드에 접근하는 것이기 때문이다.
d = D()
d.ping()    
"""
D의 ping은 메서드를 두 번 호출한다. 
1. super().ping() -> A.ping()
2. 그 다음 호출한 메세지 -> D.ping()
"""

ping: <__main__.D object at 0x105f73ee0>
post-ping: <__main__.D object at 0x105f73ee0>


'\nD의 ping은 메서드를 두 번 호출한다. \n1. super().ping() -> A.ping()\n2. 그 다음 호출한 메세지 -> D.ping()\n'

In [66]:
from diamond import D
d = D()
d.pingpong()

ping: <diamond.D object at 0x105f73100>
post-ping: <diamond.D object at 0x105f73100>
ping: <diamond.D object at 0x105f73100>
pong: <diamond.D object at 0x105f73100>
pong: <diamond.D object at 0x105f73100>


        ping: <diamond.D object at 0x105f73a90>     -> self.ping으로 D.ping()이다.  
        post-ping: <diamond.D object at 0x105f73a90>-> super().ping()으로 A.ping()   
        ping: <diamond.D object at 0x105f73a90>     -> self.pong()dmfh MRO에 따라 B.pong()  
        pong: <diamond.D object at 0x105f73a90>     -> super().pong()으로 MRO에 따라 B.pong()  
        pong: <diamond.D object at 0x105f73a90>     -> C.pong(self)가 출력한 것으로 MRO를 무시하고 C.pong()을 호출

MRO는 상속 그래프 뿐ㄴ만 아니라 서브클래스 정의에 나열된 슈퍼클래스들의 순서도 고려한다. `class D(B, C)` 를 `class D(C, B)`로 바꾸면 C를 먼저 찾도록 변경됨.  
필자는 계층구조를 조사할 때 MRO를 종종 살펴본다

In [67]:
#[12-8] 여러 클래스에서 __mro__ 속성 조사하기
bool.__mro__

(bool, int, object)

In [2]:
def print_mro(cls):
    print(', '.join(c.__name__ for c in cls.__mro__))

In [73]:
import numbers
import io

print_mro(bool)
# print_mros(FrenchDeck)
print_mro(int)
print_mro(numbers.Integral)
print_mro(io.BytesIO)
print_mro(io.TextIOWrapper)

bool, int, object
int, object
Integral, Rational, Real, Complex, Number, object
BytesIO, _BufferedIOBase, _IOBase, object
TextIOWrapper, _TextIOBase, _IOBase, object


In [74]:
import tkinter

print_mro(tkinter.Text)

Text, Widget, BaseWidget, Misc, Pack, Place, Grid, XView, YView, object


## 12.3 실세계에서의 다중 상속
다중상속을 잘 활용할 수도 있. [디자인 패턴] 중 어댑터 패턴은 다중 상속을 사용하므로, 분명 다중 상속이 잘못된 것은 아니다. (그 책에서 설명하는 나머지 22개 패턴은 단일상속을 이용함)  
파이썬 표준 라이브러리에서는 collection.abc 패키지에서 다중 상속을 많이 이용한다. 그렇다고 논란거리가 되는 것은 아님.  
Tkinter는 오래되었지만, 다음 절에서 설명하기 좋은 패키지이다.

In [3]:
import tkinter
print_mro(tkinter.Toplevel)     # 최상위 윈도우 클래스
print_mro(tkinter.Widget)       # 윈도우에 위치시킬 수 있는 모든 시각 객체의 슈퍼클래스
print_mro(tkinter.Button)       # 평범한 버튼 클래스
print_mro(tkinter.Entry)        # 편집할 수 있는 한 줄짜리 텍스트 필드
print_mro(tkinter.Text)         # 편집할 수 있는 여러 줄짜리 텍스트 필드

Toplevel, BaseWidget, Misc, Wm, object
Widget, BaseWidget, Misc, Pack, Place, Grid, object
Button, Widget, BaseWidget, Misc, Pack, Place, Grid, object
Entry, Widget, BaseWidget, Misc, Pack, Place, Grid, XView, object
Text, Widget, BaseWidget, Misc, Pack, Place, Grid, XView, YView, object


Toplebel은 시각적 위젯 중 유일하게 Widget에서 상속하지 않는다.

## 12.4 다중 상속 다루기
1. 인터페이스 상속과 구현 상속을 구분한다.